# BigQuery - BQML Online Predictions

This notebook export the BigQuery ML model create in `02 - BigQuery - BQML` and then uses AI Platform to upload the model and deploy it to and endpoint for online predictions.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`
- `02 - BigQuery - BQML`

**Overview**

<img src="architectures/statmike-mlops-03.png">

---
## Setup

Setup Parameters For Model Deployment

In [215]:
PROJECT_ID='statmike-mlops'
REGION='us-central1'

MODEL_NAME='MODEL_BQML-DIGITS'
ENDPOINT_NAME='ENDPOINT_BQML-DIGITS'
MODEL_DIR='gs://{}/digits/bqml'.format(PROJECT_ID)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

params = {"MODEL_DIR":MODEL_DIR}
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'
DEPLOY_COMPUTE='n1-standard-4'

Setup AI Platform Python Clients
- https://googleapis.dev/python/aiplatform/latest/index.html

In [216]:
from google.cloud import aiplatform

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

---
## Export the BigQuery Model

Export the BigQuery Model:
- https://cloud.google.com/bigquery-ml/docs/exporting-models

In [217]:
%%bigquery --params $params
EXPORT MODEL `statmike-mlops.digits.digits_lr`
OPTIONS(URI = @MODEL_DIR)

""


---
## Upload the Model to AI Platform

Import the Model:
- https://cloud.google.com/ai-platform-unified/docs/predictions/importing-custom-trained-model

Create a client to the Model Service, define the Model, and upload the model:

In [218]:
clients['model'] = aiplatform.gapic.ModelServiceClient(client_options=client_options)

MODEL = {
    "display_name": MODEL_NAME,
    "metadata_schema_uri": "",
    "artifact_uri": MODEL_DIR,
    "container_spec": {
        "image_uri": DEPLOY_IMAGE,
        "command": [],
        "args": [],
        "env": [],
        "ports": [{"container_port": 8080}],
        "predict_route": "",
        "health_route": ""
    }
}

uploaded_model = clients['model'].upload_model(parent=PARENT, model=MODEL)

Retrieve the model information and view the name and display name:

In [219]:
model_info = clients['model'].get_model(name=uploaded_model.result(timeout=180).model)
model_info.display_name, model_info.name

('MODEL_BQML-DIGITS',
 'projects/691911073727/locations/us-central1/models/314636247404380160')

---
## Create the AI Platform Endpoint

Deploy the Model:
- https://cloud.google.com/ai-platform-unified/docs/predictions/deploy-model-api

Create a client to the Endpoint Service and use it to create the endpoint:

In [220]:
clients['endpoint'] = aiplatform.gapic.EndpointServiceClient(client_options=client_options)

endpoint = clients['endpoint'].create_endpoint(parent=PARENT, endpoint={"display_name": ENDPOINT_NAME})

Retrieve the endpoint information and view the name and display name:

In [221]:
endpoint_info = clients['endpoint'].get_endpoint(name=endpoint.result(timeout=180).name)
endpoint_info.display_name, endpoint_info.name

('ENDPOINT_BQML-DIGITS',
 'projects/691911073727/locations/us-central1/endpoints/4801558482404769792')

---
## Deploy the Model to the AI Platform Endpoint

In [222]:
DMODEL = {
        "model": model_info.name,
        "display_name": 'DEPLOYED_'+MODEL_NAME,
        "dedicated_resources": {
            "min_replica_count": 1,
            "max_replica_count": 1,
            "machine_spec": {
                    "machine_type": DEPLOY_COMPUTE,
                    "accelerator_count": 0,
                }
        }   
}

TRAFFIC = {
    '0' : 100
}

dmodel = clients['endpoint'].deploy_model(endpoint=endpoint_info.name, deployed_model=DMODEL, traffic_split=TRAFFIC)

Retrieve the deployed model information from the endpoint:

In [224]:
clients['endpoint'].get_endpoint(name=endpoint_info.name).deployed_models

[id: "7223861763232497664"
model: "projects/691911073727/locations/us-central1/models/314636247404380160"
display_name: "DEPLOYED_MODEL_BQML-DIGITS"
create_time {
  seconds: 1618276264
  nanos: 863285000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
]

---
## Predictions

Get an online prediction:
- https://cloud.google.com/ai-platform-unified/docs/predictions/online-predictions-custom-models#online_predict_custom_trained-python

Create a client to the prediction service:

In [225]:
clients['prediction'] = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

Setup an observation for prediction:

In [226]:
%%bigquery pred
SELECT *
FROM `statmike-mlops.digits.digits_prepped`
WHERE splits='TEST'

In [227]:
pred.head(1)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p57,p58,p59,p60,p61,p62,p63,target,target_OE,SPLITS
0,0.0,0.0,0.0,0.0,13.0,16.0,6.0,0.0,0.0,0.0,...,0.0,0.0,1.0,12.0,14.0,5.0,0.0,1,Odd,TEST


In [228]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]
#newob

Request preciction from the prediction service:

In [229]:
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

response = clients['prediction'].predict(endpoint=endpoint_info.name, instances=[json_format.ParseDict(newob, Value())], parameters=json_format.ParseDict({}, Value()))

Review the prediction response (note the response is in protobuf format):

In [194]:
#response.predictions._pb

In [232]:
response.predictions._pb[0].struct_value.fields['predicted_target'].list_value.values[0].string_value

'1'

---
## Remove Resources
- undeploy-model
- remove endpoint
- remove model
- delete model files

Undeploy Model:

In [233]:
dmodel = clients['endpoint'].get_endpoint(name=endpoint_info.name).deployed_models[0].id
clients['endpoint'].undeploy_model(endpoint=endpoint_info.name, deployed_model_id=dmodel)

Delete Endpoint:

In [234]:
clients['endpoint'].delete_endpoint(name=endpoint_info.name)

Remove Model:

In [235]:
clients['model'].delete_model(name=model_info.name)

Delete Model Files:

In [236]:
from google.cloud import storage
gcs = storage.Client()

path = gcs.bucket(PROJECT_ID)
blobs = path.list_blobs(prefix='digits/bqml')
for blob in blobs:
    blob.delete()